# Molecular dating of Holocephali using fossil calibrations


## 1. Introduction

### 1.1 L'article
Grâce à la leur position à la base des poissons __osseux__, les poissons __cartilagineux__ sont considérés comme un excellent _outgroup_ dans les études comparatives des vertebrés.
<br/>
Récemment, le génome de _Callorhinchus milii_ (__chimère éléphant__) a été proposé comme génome modèle pour comprendre l'histoire évolutive des génomes vertebrés grâce sa position phylogénétique et à son génome de petite taille (910 Mb). Malgré un tel intérêt croissant, l’histoire évolutive des _holophaceans_ modernes ainsi que leur rélation avec _Elasmobranchii_ (sous-classe de _Chondrichthyes_, qui regroupe les requins, raies et chimères) et d'autres vertebrés à machoire n'a pas été bien documentée à cause de l'absence de matériaux fossiles bien conservés.
<br/><br/>
Dans cette étude, les auteurs ont assemblé le génome mitchondrial de 8 représentants de trois familles d'_holophaceans_ modernes, pour pouvoir étudier leurs rélations phylogénétiques et histoire évolutive. Pour faire cela, ils ont analysé par __maximum__ de __vraisemblance__ les séquences alignées sans ambiguïté de ces _holophaceans_ avec 17 autres vertebrés (9,409 positions nucléotidiques en excludant la troisième position du codon).
<br/>
L'arbre généré supporte l'hypothèse d'une seule origine des _holophaceans_ modernes et leur relation avec le groupe-soeur des _Elasmobranchii_. 
L’arbre mitogénomique a récupéré le plus grand nombre de _Callorhinchus_ basaux dans les _Chimaeriformes_, lequel est groupe-soeur avec les clades des deux familles restantes (_Rhinochimaeridae_ et _Chimaeridae_). 
<br/>
L’arbre temporel dérivé d’une horloge moléculaire détendue bayésienne suggère que les _holophaceans_ sont originaires du __Silurien__ vers 420 Ma, ayant survécu dans la fin du __Permien__ à l'extinction de masse et qui ont subi une diversification familiale de la fin du __Jurassique__ au début du __Crétacé__. Ce scénario évolutif proposé est bien en accord avec celui basé sur les observations paléontologiques.
<br/><br/>
Source: [Evolutionary Origin and Phylogeny of the Modern Holocephalans (Chondrichthyes: Chimaeriformes): A Mitogenomic Perspective](https://academic.oup.com/mbe/article/27/11/2576/1121974?login=true)

### 1.2 Le projet
La datation d’une phylogénie repose sur l’interprétation de la divergence entre les séquences à la lumière des calibrations fossiles, et à la lumière des _a priori_ sur ces calibrations et sur la forme de la phylogénie. <br/>
Il s’agit d’un problème inférentiel difficile où les _a priori_ et les __mouvements__ qui sont utilisés peuvent avoir un impact important sur le résultat et sur la convergence du __MCMC__. Dans ce projet, on va ré-analyser un ensemble de données construit par _Inoue et al._ (cfr. __1.1 L'article__) pour évaluer la robustesse de leurs résultats par rapport aux _a priori_ ou aux __mouvements__ qui sont utilisés.

### 1.3 Les données
Comme anticipé dans la section __1.1 L'article__, les données sont constituées par des génomes mitochrondiaux entiers assemblés de 8 _holophaceans_ (répresentant les trois familles):
- _Callorhinchus milii_ (**_Callorhinchidae_**)
- _Callorhinchus capensis_ (**_Callorhinchidae_**)
- _Callorhinchus callorynchus_ (**_Callorhinchidae_**)
- _Hariotta raleighana_ (**_Rhinochimaeridae_**)
- _Rhinochimaera pacifica_ (**_Rhinochimaeridae_**)
- _Hydrolagus lemures_ (**_Chimaeridae_**)
- _Chimaera sp_ (**_Chimaeridae_**)
- _Chimaera monstrosa_ (**_Chimaeridae_**)

Et 15 autres vertebres: _Cyprinus carpio_, _Polymixia japonica_, _Tetraodon nigroviridis_, _Protopterus aethiopicus_, _Xenopus tropicalis_, _Struthio camelus_, _Ornithorhynchus anatinus_, _Homo sapiens_, _Plesiobatis daviesi_, _Okamejei kenojei_, _Amblyraja radiata_, _Squalus acanthias_, _Heterodontus francisci_, _Scyliorhinus canicula_, _Mustelus manazo_.
<br/>
Les séquences sont contenues dans le fichier `aln_prot.fst`. Il contient les séquences d'acides aminés de 12 gènes codant des protéines des génomes mitochondriaux (exclusion des gènes ARNt et ARNr).

In [12]:
# read nucleotide data
data <- readDiscreteCharacterData("../Projet/fishes/aln_prot.fst")

n_species <- data.ntaxa()
n_branches <- 2 * n_species - 2 # car on veut un arbre enraciné
taxa <- data.taxa()

# root age
root_age ~ dnUniform(0,1000)

   Successfully read one character matrix from file '../Projet/fishes/aln_prot.fst'


In [13]:
# tree is produced by a birth death process

# speciation rate
lambda <- 1.000000000000001 # it can be egal to mu, math error

# extinction rate
mu <- 1 


# sampling fraction (in the present)
rho <- 0.00046 # 23 sampled species / 50'000 vertebrates species estimated

# time tree produced by a birth-death process (BDP)
timetree ~ dnBDP(lambda, mu, rho, root_age, samplingStrategy="uniform", condition="nTaxa", taxa=taxa)

In [3]:
# read tree from file
tree <- readTrees("../Projet/fishes/vertebrate_Poisson_NJ.tree", treetype="clock")[1]

   Attempting to read the contents of file "vertebrate_Poisson_NJ.tree"
   Successfully read file


In [14]:
print(timetree)

(((((((Squalus_acanthias[&index=23]:41.200000,(Chimaera_sp[&index=22]:37.000000,Amblyraja_radiata[&index=21]:37.000000)[&index=24]:4.200000)[&index=25]:3.200000,(Polymixia_japonica[&index=20]:34.400000,Homo_sapiens[&index=19]:34.400000)[&index=26]:10.000000)[&index=27]:10.600000,Struthio_camelus[&index=18]:55.000000)[&index=28]:0.600000,(Plesiobatis_daviesi[&index=17]:42.000000,Xenopus_tropicalis[&index=16]:42.000000)[&index=29]:13.600000)[&index=30]:3.200000,((Tetraodon_nigroviridis[&index=15]:34.400000,Mustelus_manazo[&index=14]:34.400000)[&index=31]:19.400000,Chimaera_monstrosa[&index=13]:53.800000)[&index=32]:5.000000)[&index=33]:2.600000,((Callorhinchus_capensis[&index=12]:42.000000,Hydrolagus_lemures[&index=11]:42.000000)[&index=34]:13.600000,(Callorhinchus_callorynchus[&index=10]:41.600000,Ornithorhynchus_anatinus[&index=9]:41.600000)[&index=35]:14.000000)[&index=36]:5.800000)[&index=37]:27.444199,(((Okamejei_kenojei[&index=8]:42.000000,Rhinochimaera_pacifica[&index=7]:42.000000

In [15]:
# ajust terminal branch lengths so as to make the tree ultrametric
tree.makeUltrametric()

In [16]:
print(tree)

((((Squalus_acanthias[&index=23]:358.449985,((Mustelus_manazo[&index=22]:334.370325,Scyliorhinus_canicula[&index=21]:334.370325)[&index=24]:16.716051,Heterodontus_francisci[&index=20]:351.086376)[&index=25]:7.363609)[&index=26]:28.123071,((Amblyraja_radiata[&index=19]:309.863313,Okamejei_kenojei[&index=18]:309.863313)[&index=27]:60.519665,Plesiobatis_daviesi[&index=17]:370.382977)[&index=28]:16.190079)[&index=29]:38.297343,((((Chimaera_monstrosa[&index=16]:248.817676,Chimaera_sp[&index=15]:248.817676)[&index=30]:20.397855,Hydrolagus_lemures[&index=14]:269.215531)[&index=31]:18.951432,(Rhinochimaera_pacifica[&index=13]:244.560589,Hariotta_raleighana[&index=12]:244.560589)[&index=32]:43.606374)[&index=33]:14.891585,(Callorhinchus_callorynchus[&index=11]:226.250186,(Callorhinchus_capensis[&index=10]:224.968129,Callorhinchus_milii[&index=9]:224.968129)[&index=34]:1.282057)[&index=35]:76.808363)[&index=36]:121.811851)[&index=37]:18.129601,(((Tetraodon_nigroviridis[&index=8]:378.321869,Polym

In [17]:
tree.rescale( 1000 / tree.rootAge() )

# set the value of the tree topology
# timetree.setValue(tree)

In [18]:
print(tree)
timetree

((((Squalus_acanthias[&index=23]:809.142179,((Mustelus_manazo[&index=22]:754.786287,Scyliorhinus_canicula[&index=21]:754.786287)[&index=24]:37.733749,Heterodontus_francisci[&index=20]:792.520036)[&index=25]:16.622143)[&index=26]:63.483229,((Amblyraja_radiata[&index=19]:699.465717,Okamejei_kenojei[&index=18]:699.465717)[&index=27]:136.613238,Plesiobatis_daviesi[&index=17]:836.078955)[&index=28]:36.546453)[&index=29]:86.449985,((((Chimaera_monstrosa[&index=16]:561.665183,Chimaera_sp[&index=15]:561.665183)[&index=30]:46.044820,Hydrolagus_lemures[&index=14]:607.710003)[&index=31]:42.779757,(Rhinochimaera_pacifica[&index=13]:552.055506,Hariotta_raleighana[&index=12]:552.055506)[&index=32]:98.434253)[&index=33]:33.615316,(Callorhinchus_callorynchus[&index=11]:510.722766,(Callorhinchus_capensis[&index=10]:507.828733,Callorhinchus_milii[&index=9]:507.828733)[&index=34]:2.894034)[&index=35]:173.382309)[&index=36]:274.970318)[&index=37]:40.924607,(((Tetraodon_nigroviridis[&index=8]:853.999703,Po

In [19]:
clade_1 <- clade("Plesiobatis_daviesi","Mustelus_manazo")
speciation_clade_1 := tmrca(timetree, clade_1)
min_clade_1 <- 190.0
max_clade_1 <- 463.0
width_age_1 <- (max_clade_1 - min_clade_1) / 2.0
mean_age_1 <- min_clade_1 + width_age_1
# obs_age_1 ~ dnNormal(speciation_clade_1, mean_age_1) #dnUniform(speciation_clade_1 - width_age_1, speciation_clade_1 + width_age_1)

obs_age_1 ~ dnExponential(0.2, offset = - speciation_clade_1)

# ?moves sur obs_age
obs_age_1.clamp(-min_clade_1)
speciation_clade_1
mean_age_1

   58.8
   326.5


In [20]:
speciation_clade_1

   58.8


In [21]:
# we assume a relaxed molecular clock, of unknown rate
mean_clockrate ~ dnExponential(1.0)
relvar_clockrate ~ dnExponential(1.0)

alpha := 1.0 / relvar_clockrate
beta := alpha / mean_clockrate

for (i in 1:n_branches) {
    clockrate[i] ~ dnGamma(alpha, beta)
}

Q := fnMtRev()

seq ~ dnPhyloCTMC( tree=timetree, Q=Q, branchRates=clockrate, type="AA" )
# le taux d'évolution dans toutes les branches est égale au clockrate
seq.clamp( data )

my_model = model(timetree)

# moves
moves = VectorMoves()

# moves.append(mvScale(lambda, weight=1.0))
# moves.append(mvScale(mu, weight=1.0))

for (i in 1:n_branches) {
    moves.append(mvScale(clockrate[i], weight=1.0, lambda=0.5))    
}
moves.append(mvScale(relvar_clockrate, weight=1.0, lambda=0.1))
moves.append(mvScale(mean_clockrate, weight=1.0, lambda=0.1))

# moves.append(mvSlide(gamma, weight=1.0, delta=0.1))
# moves.append(mvScale(kappa, weight=1.0))
# moves.append(mvScale(clockrate, weight=1.0, lambda=0.1))

moves.append(mvNarrow(timetree, weight=10))
moves.append(mvFNPR(timetree, weight=10))
moves.append(mvScale(root_age, weight=1.0, lambda=0.1))
moves.append(mvNodeTimeSlideUniform(timetree, weight=20)) # bouge un seul noeud
moves.append(mvSubtreeScale(timetree, weight=4)) # bouge de façon proportionnel tous les noeuds d'un sous-arbre

monitors[1] = mnModel(filename="../Projet/analyses/fish_clock_1.log", printgen=10, separator = TAB)
monitors[2] = mnFile(timetree ,filename="../Projet/analyses/fish_clock_1.trees", printgen=10, separator = TAB)
monitors[3] = mnScreen(printgen=100, mean_clockrate, relvar_clockrate, root_age, obs_age_1)

# Make the analysis object
analysis = mcmc(my_model, monitors, moves)
# Run the MCMC
analysis.run(5000)

treetrace = readTreeTrace("../Projet/analyses/fish_clock_1.trees", treetype="clock", burnin=0.1)
map_tree = mccTree(treetrace, "../Projet/analyses/fish_clock_1.tree")

   Could not compute lnProb for node obs_age_1.
   -190
   
   
   Drawing new initial states ... 
   
   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 51 different moves in a random move schedule with 91 moves per iteration
   

Iter        |      Posterior   |     Likelihood   |          Prior   |   mean_clock..   |      obs_age_1   |   relvar_clo..   |       root_age   |    elapsed   |        ETA   |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0           |        -187175   |        -187114   |       -60.7878   |     0.04089485   |           -190   |      0.4013955   |       417.3057   |   00:00:00   |   --:--:--   |
100         |       -44394.3   |       -44352.9   |       -41.3555   |      0.0242642   |           -190   |      0.6497356   |       207.9786   |   00:01:08   |   --:--:--   |
200         |

Iter        |      Posterior   |     Likelihood   |          Prior   |   mean_clock..   |      obs_age_1   |   relvar_clo..   |       root_age   |    elapsed   |        ETA   |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4000        |       -43993.6   |         -43966   |       -27.6326   |     0.01910034   |           -190   |       1.489695   |       191.0905   |   00:36:02   |   00:09:00   |
4100        |       -43550.5   |       -43512.4   |       -38.0593   |     0.02049838   |           -190   |       1.266352   |       198.1725   |   00:36:58   |   00:08:06   |
4200        |       -43535.3   |       -43503.1   |       -32.1591   |     0.02195686   |           -190   |       1.831092   |       190.0694   |   00:37:46   |   00:07:11   |
4300        |       -43540.7   |       -43505.6   |       -35.0393   |     0.02177625   |           -190   |       